In [1]:
import pandas as pd 
import numpy as np

In [184]:
ticket_dat = pd.read_csv('cleaned_translated.csv')
FAQ_dat = pd.read_csv('FAQ_translation.csv')
print('Ticket Data: {} Rows'.format(ticket_dat.shape[0]))
print('FAQ Data: {} Rows'.format(FAQ_dat.shape[0]))

Ticket Data: 18288 Rows
FAQ Data: 389 Rows


In [3]:
# Print Columns in Ticket Data
ticket_dat.columns.values

array(['tn', 'title', 'sensitiv', 'Queue', 'name', 'create_time', 'id',
       'ticket_id', 'article_type_id', 'article_sender_type_id', 'a_from',
       'a_reply_to', 'a_to', 'a_cc', 'a_subject', 'a_message_id',
       'a_content_type', 'a_body', 'incoming_time', 'content_path',
       'valid_id', 'create_time1', 'create_by', 'change_time',
       'change_by', 'a_in_reply_to', 'a_references', 'a_message_id_md5',
       'a_body_clean', 'lang', 'translation'], dtype=object)

In [4]:
# Print Columns in FAQ Data
FAQ_dat.columns.values

array(['ques_id', 'language', 'ques', 'ansTitle', 'ansContent',
       'translation', 'translation_title', 'ques_translation'],
      dtype=object)

# Lets Settle for A Final Form of the Data
## Tickets
ticket_no, article_id (with prefix for ticket vs FAQ), content_original, content_clean, lang, content_translate

## FAQs
ques_id (prefix for FAQ question), ans_id (prefix for FAQ ans), ansTitle_cleaned, ansContent_cleaned, ansTitle_translate, ansContent_translate, quesContent_cleaned, quesContent_translate




# Ticket Data

# Removing Some Rows and Final Preprocessing

First remove non english / originally german enteries from ticket data (minor loss of data). Remember though, some "conversations" lost as a result.

In [5]:
ticket_dat.groupby('lang').count().tn

lang
ar           1
co           3
cs           1
da           7
de       14498
el           1
en        3738
fi           2
fr           7
gd           3
haw          1
hi           1
hmn          1
id           1
it           1
ku           1
lb           8
no           3
pl           3
ro           1
sk           1
sl           1
tr           1
zh-CN        3
Name: tn, dtype: int64

In [6]:
ticket_dat = ticket_dat[ticket_dat.lang.isin(['en', 'de'])]
print('{} Rows Remain'.format(ticket_dat.shape[0]))

18236 Rows Remain


Remove data with NaN in the content


In [7]:
print('{} NaN enteries removed'.format(pd.isnull(ticket_dat.a_body_clean).sum()))
ticket_dat = ticket_dat[~pd.isnull(ticket_dat.a_body_clean)]

32 NaN enteries removed


## Concatenate All Ticket Questions / Answers within an Exchange

To get tickets into a Question and Answer Format we are going to make the (probably not amazingly well justified) provisional assumption that we can concatnate all user messages in the ticket as a question and all it team parts of the ticket as a question. Language is just set to max over language strings which will return en over de (alphabetical).

In [118]:
ticket_dat['article_sender_type_id'] = ticket_dat.article_sender_type_id.apply(lambda x: 'question' if x ==3 else 'answer')

In [154]:
grouped_tickets = ticket_dat.groupby(['tn', 'article_sender_type_id']).agg(
    {'translation':'sum', 
     'lang':'max'}).unstack()

In [155]:
coltitle = list(grouped_tickets.columns.get_level_values(0))
subcoltitle = list(grouped_tickets.columns.get_level_values(1))

grouped_tickets.columns = ["_".join(x) for x in zip(subcoltitle, coltitle)]
grouped_tickets = grouped_tickets.reset_index()

In [156]:
grouped_tickets.head()

,tn,answer_translation,question_translation,answer_lang,question_lang
0,18014685,"Dear Mr. Daniel Aeppli Fortunately, you have a...","Dear Ladies and Gentlemen, I still have about ...",de,de
1,18022254,VPN is blocked 07/02/2018 09:25 - Elmar Sales ...,Dear Pelyuan Zhang\n\nWe contact you because o...,en,en
2,18023005,Dear Yargo C Bonetti Basically NETHZ guest acc...,"Dear Ladies and Gentlemen, we have users (eg s...",de,de
3,18023224,Dear Mrs. Zumsteg Thank you for your e-mail. P...,Dear Sir / Madam I wanted to download from the...,de,de
4,18024487,Dear Ms Barbara Loepfe We can enter an address...,Hello We received a new toner for our printer ...,de,de


Quickly check if Q and A were always in the same language. Answer: not always but they should've all been translated anyways

In [173]:
(grouped_tickets.answer_lang + "_"+grouped_tickets.question_lang).value_counts()

de_de    3084
en_en     825
de_en     216
en_de     115
dtype: int64

Now we need to pivot the table so we have a question and answer column and a ticket number.

## Rename some Columns

In [157]:
grouped_tickets.columns.values

array(['tn', 'answer_translation', 'question_translation', 'answer_lang',
       'question_lang'], dtype=object)

In [176]:
ticket_column_rename={"tn": "ticket_id", "answer_translation": "answer", "question_translation": "question"}

grouped_tickets.rename(index=str, columns = ticket_column_rename, inplace=True)

## Drop Some Columns

In [177]:
ticket_columns_to_drop = []

grouped_tickets.drop(ticket_columns_to_drop, axis=1, inplace=True)

In [179]:
grouped_tickets.head()

,ticket_id,answer,question,answer_lang,question_lang
0,18014685,"Dear Mr. Daniel Aeppli Fortunately, you have a...","Dear Ladies and Gentlemen, I still have about ...",de,de
1,18022254,VPN is blocked 07/02/2018 09:25 - Elmar Sales ...,Dear Pelyuan Zhang\n\nWe contact you because o...,en,en
2,18023005,Dear Yargo C Bonetti Basically NETHZ guest acc...,"Dear Ladies and Gentlemen, we have users (eg s...",de,de
3,18023224,Dear Mrs. Zumsteg Thank you for your e-mail. P...,Dear Sir / Madam I wanted to download from the...,de,de
4,18024487,Dear Ms Barbara Loepfe We can enter an address...,Hello We received a new toner for our printer ...,de,de


# FAQ Data

In [185]:
FAQ_dat.head()

,ques_id,language,ques,ansTitle,ansContent,translation,translation_title,ques_translation
0,8502,EN,Received a phishing mail?,Phishing Mail warning,This is a phishing mail. You should delete it....,This is a phishing mail. You should delete it....,Phishing Mail warning,Received a phishing mail?
1,8503,EN,Lockout on Account,My account is locked,You had a lockout on your account because you ...,You had a lockout on your account because you ...,My account is locked,Lockout on Account
2,8506,EN,Blocking mailsender,Block a mailsender,You can add the sender on your personal blackl...,You can add the sender on your personal blackl...,Block a mailsender,Blocking mailsender
3,8509,EN,Credit overdrawn,I can't print anymore,Your credit is 6.90.- in minus. You have to lo...,Your credit is 6.90.- in minus. You have to lo...,I can't print anymore,Credit overdrawn
4,8509,EN,Credit overdrawn,Why is my account in minus,The reason why your account is minus is the fo...,The reason why your account is minus is the fo...,Why is my account in minus,Credit overdrawn


In [186]:
# Rename Columns First
faq_column_rename={"ques_id": "faq_id", "language": "lang", "translation": "answer", 
                      "ques_translation": "question", "translation_title": "answer_title"}

FAQ_dat.rename(index=str, columns = faq_column_rename, inplace=True)

In [188]:
# Drop non english columns (not used later at all. 
# If we want to inspect original language and translation should come back here)

faq_columns_to_drop = ['ansTitle', 'ansContent', 'ques']

FAQ_dat.drop(faq_columns_to_drop, axis=1, inplace=True)

In [189]:
# Make the langauge lower case 
FAQ_dat['lang'] = FAQ_dat.lang.str.lower()

In [193]:
FAQ_dat.head()

,faq_id,lang,answer,answer_title,question
0,8502,en,This is a phishing mail. You should delete it....,Phishing Mail warning,Received a phishing mail?
1,8503,en,You had a lockout on your account because you ...,My account is locked,Lockout on Account
2,8506,en,You can add the sender on your personal blackl...,Block a mailsender,Blocking mailsender
3,8509,en,Your credit is 6.90.- in minus. You have to lo...,I can't print anymore,Credit overdrawn
4,8509,en,The reason why your account is minus is the fo...,Why is my account in minus,Credit overdrawn


# Final Two Datasets

## Ticket

In [197]:
grouped_tickets.head(10)

,ticket_id,answer,question,answer_lang,question_lang
0,18014685,"Dear Mr. Daniel Aeppli Fortunately, you have a...","Dear Ladies and Gentlemen, I still have about ...",de,de
1,18022254,VPN is blocked 07/02/2018 09:25 - Elmar Sales ...,Dear Pelyuan Zhang\n\nWe contact you because o...,en,en
2,18023005,Dear Yargo C Bonetti Basically NETHZ guest acc...,"Dear Ladies and Gentlemen, we have users (eg s...",de,de
3,18023224,Dear Mrs. Zumsteg Thank you for your e-mail. P...,Dear Sir / Madam I wanted to download from the...,de,de
4,18024487,Dear Ms Barbara Loepfe We can enter an address...,Hello We received a new toner for our printer ...,de,de
5,18025134,"Hi Aleks, I could find time. But first idtt.et...","Hello Peter, Do you have time to update to idt...",de,de
6,18028816,Sali Tommy The mailbox has been converted. You...,"Description: Hello, Could you please convert t...",de,de
7,18030232,Dear Prof. Dr. Emilie Thomassot\n \nOf course ...,"Hi,\nMy name is Emilie Thomassot and I’m a gue...",en,en
8,18030825,Dear Dr. Sebastiano Rossi\n \nYour vpn service...,Dear ETH support\n\nLately my login for the et...,en,en
9,18031632,Dear Doctor. Matteo Facchini Your request has ...,"Good evening, I was exmatriculated on 8.1.2018...",de,de


## FAQ

In [195]:
FAQ_dat.head(10)

,faq_id,lang,answer,answer_title,question
0,8502,en,This is a phishing mail. You should delete it....,Phishing Mail warning,Received a phishing mail?
1,8503,en,You had a lockout on your account because you ...,My account is locked,Lockout on Account
2,8506,en,You can add the sender on your personal blackl...,Block a mailsender,Blocking mailsender
3,8509,en,Your credit is 6.90.- in minus. You have to lo...,I can't print anymore,Credit overdrawn
4,8509,en,The reason why your account is minus is the fo...,Why is my account in minus,Credit overdrawn
5,8528,en,I've reset your permissions. Can you please tr...,Reset NAS permissions,Reset NAS permissions
6,8511,en,The settings for the ETH VPN are as follows:Se...,VPN settings of the ETH,Settings for VPN
7,8507,en,You can add the sender on your personal whitel...,Mail blocked mistakenly,Mail blocked mistakenly
8,8511,en,You can find all the informations for the VPN ...,Installation VPN Client,Settings for VPN
9,8532,en,Please download first all software local to y...,Software installation from IDES,How can I install software from IDES?


# Export to file

In [ ]:
grouped_tickets.to_csv('../data/ticket_dat.csv')
FAQ_dat.to_csv('../data/faq_dat.csv')